In [1]:
%pip install wandb
%pip install tensorflow
%pip install keras
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import wandb
from wandb.keras import WandbMetricsLogger
import matplotlib.pyplot as plt

In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: spbrady18 (tool-lab-3). Use `wandb login --relogin` to force relogin


True

In [4]:
checkpointCallbackFunction =  tf.keras.callbacks.ModelCheckpoint(
                                filepath='/checkpoints/model.{epoch:02d}-{val_loss:.2f}.h5',
                                save_freq="epoch")

In [15]:
# get your data
(x_train, y_train), (x_val, y_val) = keras.datasets.cifar100.load_data()

# constant values
NUM_CLASSES = 100 #100 prediction classes
INPUT_SHAPE = (32,32,3) #shape of the input image 32x32 with 3 channels

# hyperparameters you will be tuning
BATCH_SIZE = 50 
EPOCHS = 25
LEARNING_RATE = 1e-3
L1NF = 16
FDROPOUT = 0.5

wandb.init(project = 'Tool Lab 4',
          config={
              'bs':BATCH_SIZE,
              'lr':LEARNING_RATE,
              'epochs': EPOCHS,
              'l1nf':L1NF,
              'fdr':FDROPOUT
          },
           name='RUN 20: random rotations, 3 pooling, 3x L1NF 16, (Adam optomizer), batch normalization x3, once before all' #this is your run name, please number your runs 0-n and tell what you changed
          )

# onehot encode your labels so your model knows its a category
y_train = tf.one_hot(y_train,
                     depth=y_train.max() + 1,
                     dtype=tf.float64)
y_val = tf.one_hot(y_val,
                   depth=y_val.max() + 1,
                   dtype=tf.float64)
  
y_train = tf.squeeze(y_train)
y_val = tf.squeeze(y_val)

# here is a basic model that you will add to
model = tf.keras.models.Sequential([
                  
                  layers.RandomFlip('horizontal'),
                  layers.RandomRotation(0.1),
                  layers.BatchNormalization(),
                  layers.Conv2D(L1NF, (3, 3), input_shape = INPUT_SHAPE,
                                activation='relu',
                                padding='same'),
                  layers.BatchNormalization(),
                  layers.MaxPooling2D(2,2),
                  layers.Conv2D(L1NF, (3, 3), input_shape = INPUT_SHAPE,
                                activation='relu',
                                padding='same'),
                  layers.BatchNormalization(),
                  layers.MaxPooling2D(2,2),

                  layers.Conv2D(1024, 3, padding="same", activation="relu"),

                  layers.Dropout(FDROPOUT),
                  
                  # DO NOT CHANGE THESE. They should be at the end of your model
                  layers.Flatten(),
                  layers.Dense(NUM_CLASSES, activation='softmax')])


# feel free to experiment with this
model.compile(loss=tf.keras.losses.binary_crossentropy,
             optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
             # DO NOT CHANGE THE METRIC. This is what you will be judging your model on
             metrics=['accuracy'],)

# here you train the model using some of your hyperparameters and send the data
# to weights and biases after every batch            
history = model.fit(x_train, y_train,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[
                      WandbMetricsLogger(log_freq='batch'),
                      checkpointCallbackFunction,
                    ])

Epoch 1/25
1000/1000 [==============================] - 128s 127ms/step - loss: 0.0461 - accuracy: 0.1829 - val_loss: 0.0399 - val_accuracy: 0.2813
Epoch 2/25
1000/1000 [==============================] - 128s 128ms/step - loss: 0.0399 - accuracy: 0.2768 - val_loss: 0.0378 - val_accuracy: 0.3336
Epoch 3/25
1000/1000 [==============================] - 130s 130ms/step - loss: 0.0377 - accuracy: 0.3184 - val_loss: 0.0367 - val_accuracy: 0.3515
Epoch 4/25
1000/1000 [==============================] - 135s 135ms/step - loss: 0.0361 - accuracy: 0.3505 - val_loss: 0.0350 - val_accuracy: 0.3803
Epoch 5/25
1000/1000 [==============================] - 128s 128ms/step - loss: 0.0347 - accuracy: 0.3777 - val_loss: 0.0347 - val_accuracy: 0.3884
Epoch 6/25
1000/1000 [==============================] - 121s 121ms/step - loss: 0.0337 - accuracy: 0.3976 - val_loss: 0.0335 - val_accuracy: 0.4081
Epoch 7/25
1000/1000 [==============================] - 122s 122ms/step - loss: 0.0327 - accuracy: 0.4175 - val_

In [13]:
#This will save the model 
model.save("trainedModel.h5")

#This will load my model
# model = keras.models.load_model("myfile/path/trainedMode.h5")